In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [3]:
url = "https://www.ons.gov.uk/file?uri=/economy/nationalaccounts/balanceofpayments/datasets/uktradecountrybycommodityimports/current/countrybycommodityimports11032021144801.zip"


In [5]:
r = requests.get(url) 

brexit_Jan21 = zf.ZipFile(io.BytesIO(r.content))

brexit_Jan21.extractall(".\\data")

filename = brexit_Jan21.namelist()

In [131]:
df = pd.read_excel(".\\data" + "\\" + filename[0], sheet_name= "Country by Commodity Import")

In [132]:
df.head()

,COMMODITY,COUNTRY,DIRECTION,1997JAN,1997FEB,1997MAR,1997APR,1997MAY,1997JUN,1997JUL,...,2020APR,2020MAY,2020JUN,2020JUL,2020AUG,2020SEP,2020OCT,2020NOV,2020DEC,2021JAN
0,0 Food & live animals,AD Andorra,IM Imports,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0 Food & live animals,AE United Arab Emirates,IM Imports,0.27,0.07,2.51,22.37,5.06,0.23,0.61,...,1.21,8.72,1.24,1.45,1.59,0.84,1.01,1.47,1.18,2.19
2,0 Food & live animals,AF Afghanistan,IM Imports,0.06,0.07,0.00,0.23,0.05,0.05,0.00,...,0.00,0.00,0.00,0.00,0.02,0.11,0.04,0.12,0.10,0.11
3,0 Food & live animals,AG Antigua & Barbuda,IM Imports,0.02,0.12,0.00,0.00,0.04,0.01,0.00,...,0.00,0.01,0.06,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0 Food & live animals,AI Anguilla,IM Imports,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [133]:
df = df.melt(id_vars = ["COMMODITY", "COUNTRY", "DIRECTION"])

df.drop("DIRECTION", axis = 1, inplace = True)

df.variable= pd.to_datetime(df.variable, format="%Y%b")

df.rename({"variable":"time", "value": "imports"},inplace = True, axis = 1)

df.COUNTRY = df.COUNTRY.str[3:]

df.imports = df.imports*1000000

In [134]:
grp = df[df.COMMODITY == 'T Total'].groupby("COUNTRY")

test = grp.agg({"imports": "sum"})

In [135]:
topcntry = test.sort_values(by = "imports", ascending = False).iloc[0:23].index.to_list()

In [136]:
df = df[df.COUNTRY.isin(topcntry)]

In [150]:
df.COMMODITY.replace({"T Total": "Total"}, inplace = True)
df.COUNTRY.replace({"United States inc Puerto Rico": "United States"}, inplace = True)

In [151]:
df.tail()

,COMMODITY,COUNTRY,time,imports
8561566,Total,Russia,2021-01-01,5.620000e+08
8561572,Total,Sweden,2021-01-01,4.300000e+08
8561599,Total,Turkey,2021-01-01,7.510000e+08
8561607,Total,United States,2021-01-01,2.333000e+09
8561617,Total,Whole world,2021-01-01,3.151300e+10


In [152]:
df.set_index(["COUNTRY", "COMMODITY", "time"], inplace = True)

In [153]:
df.head()

,,,imports
COUNTRY,COMMODITY,time,
Total EU(28),0 Food & live animals,1997-01-01,673940000.0
Belgium,0 Food & live animals,1997-01-01,44340000.0
Canada,0 Food & live animals,1997-01-01,14320000.0
Switzerland,0 Food & live animals,1997-01-01,4110000.0
China,0 Food & live animals,1997-01-01,6530000.0


In [154]:
out_file = ".\\data"+ "\\UK-imports.parquet"

pq.write_table(pa.Table.from_pandas(df.sort_index()), out_file)

In [155]:
file = ".\\data"+ "\\UK-imports.parquet"

foo = pq.read_table(file).to_pandas()

In [156]:
foo.head()

imports
COUNTRY COMMODITY             time                  
Belgium 0 Food & live animals 1997-01-01  44340000.0
                              1997-02-01  44430000.0
                              1997-03-01  43820000.0
                              1997-04-01  58810000.0
                              1997-05-01  57800000.0

In [130]:
foo.loc[("Total EU(28)", "T Total"),:]

,imports
time,
1997-01-01,8.292000e+09
1997-02-01,8.230000e+09
1997-03-01,8.649000e+09
1997-04-01,9.120000e+09
1997-05-01,8.229000e+09
...,...
2020-09-01,2.106300e+10
2020-10-01,2.204900e+10
2020-11-01,2.335000e+10


In [182]:
test = ["foo", "foobar", "chase"]

In [164]:
test1 = test;

test = 

In [171]:
test[0] = test[-1]

In [185]:
test.remove("chase")

In [186]:
test.insert(0,"chase")

In [187]:
test

['chase', 'foo', 'foobar']